In [3]:
from tensorflow.keras import models, layers, backend
import tensorflow as tf
import tensorflow_addons as tfa
from pathlib import Path
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import os, math, random, imagesize

In [ ]:
# GLOBALS
DATA_DIR = "data/"
IMAGE_SIZE = 256 
IMAGE_SHAPE = (IMAGE_SIZE, IMAGE_SIZE)
CHANNELS = 3
BATCH_SIZE = 32
EPOCHS = 30
IMG_TENSOR_SHAPE = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)

BATCHED_DATASET = tf.keras.preprocessing.image_dataset_from_directory(DATA_DIR, shuffle=True,  image_size=IMAGE_SHAPE, batch_size=BATCH_SIZE)
NUM_BATCHES = len(BATCHED_DATASET)

FILE_PATHS = BATCHED_DATASET.file_paths
CLASS_NAMES = BATCHED_DATASET.class_names
NUM_CLASSES = len(CLASS_NAMES)

In [ ]:
def get_partitioned_tf_dataset(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=43)
    train_set = ds.take(round(NUM_BATCHES * train_split))
    val_set = ds.skip(len(train_set)).take(round(len(ds) * val_split))
    test_set = ds.skip(len(train_set)+len(val_set)).take(-1)
    return train_set, val_set, test_set

In [ ]:
train_set, val_set, test_set = get_partitioned_tf_dataset(dataset)

In [ ]:
train_set = train_set.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_set = val_set.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_set = test_set.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
resize_and_scale_layer = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
    layers.experimental.preprocessing.Rescaling(1.0/255)
])

data_augmentation_layer = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2)
])

In [1]:
def build_cnn(name="CNN", image_shape=None):
    backend.clear_session()
    model = models.Sequential([
        resize_and_scale_layer,
        data_augmentation_layer,
    
        layers.Conv2D(32, (3,3), activation="relu", input_shape=image_shape),
        layers.MaxPooling2D((2,2)),
    
        layers.Conv2D(64, (3,3), activation="relu"),
        layers.MaxPooling2D((2,2)),
    
        layers.Conv2D(64, (3,3), activation="relu"),
        layers.MaxPooling2D((2,2)),
    
        layers.Conv2D(64, (3,3), activation="relu"),
        layers.MaxPooling2D((2,2)),
    
        layers.Conv2D(64, (3,3), activation="relu"),
        layers.MaxPooling2D((2,2)),
    
        layers.Conv2D(64, (3,3), activation="relu"),
        layers.MaxPooling2D((2,2)),
    
        layers.Flatten(),
        layers.Dense(64, activation="relu"),
        layers.Dense(NUM_CLASSES, activation="softmax")
    ], name="CNN")

    model.build(input_shape=image_shape)
    return model

In [ ]:
CNN = build_cnn(image_shape=IMG_TENSOR_SHAPE)
CNN.summary()

In [ ]:
def fit_and_test(model, train, val, test, batch_size=BATCH_SIZE, epochs=EPOCHS):
    history = model.fit(train, epochs=epochs, batch_size=batch_size, verbose=1, validation_data=val)
    test_score = model.evaluate(test)
    return history, test_score

In [ ]:
CNN = build_cnn(image_shape=IMG_TENSOR_SHAPE)
nadam = CNN.compile(
    optimizer="Nadam", 
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)
nadam_history, nadam_test_score = fit_and_test(CNN, train_set, val_set, test_set)

In [ ]:
# Plots the train and validation acc & loss for the model history given
def plot_train_vs_val(model_history, epochs=EPOCHS):
    plt.rcParams.update({'font.size': 16})
    plt.figure(figsize=(24,16))
    (loss, acc, val_loss, val_acc) = model_history.history.values()
    for (title, train, val, pos, loc) in [["Accuracy", acc, val_acc, "lower right", 1], ["Loss", loss, val_loss, "upper right", 2]]:
        plt.subplot(1, 2, loc)
        plt.title(f"Training vs Validation {title}")
        plt.plot(range(epochs), train, label=f"Train {title}")
        plt.plot(range(epochs), val, label=f"Val {title}")
        plt.legend(loc=pos)
        plt.grid()

In [ ]:
plot_train_vs_val(nadam_history)

In [ ]:
def predict_image(model, img):
    img_array = tf.expand_dims(tf.keras.preprocessing.image.img_to_array(img), 0)
    preds = model.predict(img_array)[0]
    pred_label = CLASS_NAMES[np.argmax(preds)]
    confidence = round(100 * (np.max(preds)), 2)
    return pred_label, confidence


def display_single_prediction_from_batched_set(batched_data):
    for images_batch, labels_batch in from_set.take(1):
        index = random.randint(0, len(images_batch)-1)
        image = images_batch[index].numpy().astype("uint8")
        actu_label = CLASS_NAMES[labels_batch[index].numpy()]
        pred_label, conf = predict_image(CNN, image)
        print(f"Predicted Label: {pred_label}")
        print(f"Actual Label:    {actu_label}")    
        print(f"Confidence:      {conf}") 
        plt.axis("off")
        plt.imshow(image)


def display_multiple_predictions_from_batched_set(batched_data, num_rows, num_cols):
    num_images = num_rows*num_cols
    plt.figure(figsize=(16, num_images*2))
    for images, labels in from_set.take(1):
        for i in range(num_images):
            pred_label, conf = predict_image(CNN, images[i].numpy())
            actual_label = CLASS_NAMES[labels[i]]
            plt.subplot(num_rows, num_cols, i+1)            
            plt.title(f"Actual: {actual_label}, \n Predicted: {pred_label}, \n Confidence: {conf}")
            plt.axis("off")
            plt.imshow(images[i].numpy().astype("uint8"))  

In [2]:
def save_h5_model(model, path="./h5_models/"):
    model_val = len(os.listdir(path))
    model.save(f"{path}{len(model_val) + 1}.h5")
    return f"Model saved as {len(model_val)}.h5"


def save_default_model(model, alt_version=None, path="models/"):
    saved_models = [float(x.split('-')[1]) for x in os.listdir(path)]
    if len(saved_models) == 0:
        model.save(f"{path}{model.name}-1.0")
    elif alt_version is not None:
        alt_version = float(math.floor(alt_version))
        if saved_models.count(alt_version) != 0:
            prevs_index = (len([x for x in saved_models if int(x) <= int(alt_version)])-1)
            prev_version = saved_models[prevs_index]
            if round(prev_version - 0.9, 1) == alt_version:
                print("Max alternate models reached, create another base version.")
            else:
                model.save(f"{path}{model.name}-{round(prev_version + .1, 1)}")
        else:
            print("New version created")
            model.save(f"{path}{model.name}-{alt_version}")
    else:
        model_version = saved_models[-1]
        model.save(f"{path}{model.name}-{float(int(model_version + 1))}")

In [ ]:
# Using ImageDataGenerator & spilt datasets
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Using split_folders python package to easily split the data
# Run the following in terminal: 
# split_folders --output split_dataset --ratio .7 .1 .2 -- data/
# After this our data is setup for the following block
split_dataset = "split_dataset"

datagen_params = dict(rescale=1./255, horizontal_flip=True, rotation_range=10)
generator_params = dict(target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, class_mode="sparse")

train_datagen = ImageDataGenerator(**datagen_params)
val_datagen = ImageDataGenerator(**datagen_params)
test_datagen = ImageDataGenerator(**datagen_params)

train_generator = train_datagen.flow_from_directory(split_dataset+"/train", **generator_params)
val_generator = val_datagen.flow_from_directory(split_dataset+"/val", **generator_params)
test_generator = test_datagen.flow_from_directory(split_dataset+"/test", **generator_params)

In [ ]:
def fit_and_test_generator(model, train_gen, val_gen, test_gen, batch_size, epochs):
    history = model.fit(train_generator, steps_per_epoch=len(train_gen), 
                        validation_data=val_generator, validation_steps=len(val_gen), 
                        batch_size=batch_size, epochs=epochs, 
                        verbose=1,
                        workers=1, use_multiprocessing=False, 
                        class_weight=None
                    )
    test_score = model.evaluate(test_gen)
    return history, test_score

In [ ]:
METRICS = [
    tf.keras.metrics.CategoricalCrossentropy(name="CategoricalCrossentropy"),
    tf.keras.metrics.CategoricalAccuracy(name="CategoricalAccuracy"),
    
    tf.keras.metrics.TruePositives(name="TruePositives"),
    tf.keras.metrics.FalsePositives(name="FalsePositives"),
    tf.keras.metrics.TrueNegatives(name="TrueNegatives"),
    tf.keras.metrics.FalseNegatives(name="FalseNegatives"),
    
    tf.keras.metrics.Precision(name="Precision"),
    tf.keras.metrics.Recall(name="Recall"),
    tf.keras.metrics.PrecisionAtRecall(0.5, name="PrecisionAtRecall"),
    tf.keras.metrics.RecallAtPrecision(0.5, name="RecallAtPrecision"),
    tf.keras.metrics.SensitivityAtSpecificity(0.5, name="SensitivityAtSpecificity"),
    tf.keras.metrics.SpecificityAtSensitivity(0.5, name="SpecificityAtSensitivity"), 
    
    tfa.metrics.F1Score(num_classes=NUM_CLASSES, threshold=0.5, name="F1Score"),
    tfa.metrics.FBetaScore(num_classes=NUM_CLASSES, threshold=0.5, name="FBetaScore"),
    tfa.metrics.MultiLabelConfusionMatrix(num_classes=NUM_CLASSES),

    tf.keras.metrics.AUC(name='AUC', from_logits=False),
    tf.keras.metrics.AUC(name='ROC', curve='ROC', from_logits=False),
    tf.keras.metrics.AUC(name='PRC', curve='PR', from_logits=False),
]

metric_names = [i.name for i in METRICS]

In [ ]:
# Working with metrics outside of accuracy because of class imbalance
BATCHED_DATASET_2 = tf.keras.preprocessing.image_dataset_from_directory(
    directory=DATA_DIR, label_mode='categorical', shuffle=True, image_size=IMAGE_SHAPE, batch_size=BATCH_SIZE
)
train_set, val_set, test_set = split_tf_dataset_train_val_test(BATCHED_DATASET_2, shuffle=True)
train_set = train_set.cache().shuffle(10000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_set = val_set.cache().shuffle(10000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_set = test_set.cache().shuffle(10000).prefetch(buffer_size=tf.data.AUTOTUNE)

CNN = build_cnn()
nadam = CNN.compile(
    optimizer="Nadam",
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=METRICS
)
nadam_history_2, nadam_test_score_2 = fit_and_test(CNN, train_set, val_set, test_set)
metric_results_2 = dict(zip(metric_names, nadam_test_score))